In [206]:
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [328]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [329]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal_u.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal.txt',header=None)

In [283]:
#print(stock_lable.head(10))

In [330]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:8999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape)
y = stock_lable.loc[0:1499,2].values
y = y.reshape(-1,1)
print(y.shape)
#x = x / 2
x = np.insert(x,0,1,axis=1)

(9000, 11)
(1500, 66)
(1500, 1)


In [331]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [332]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 1000

In [333]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.8
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[ 0.51100986  0.51848552 -0.4746211  -0.33904025  0.68943112]
 [ 1.041949   -0.4435212   0.63598022 -0.86394194  0.19830018]
 [ 0.91642987  0.12217291 -0.14872454  0.58260229  1.03943772]
 [-0.80221957 -0.6922805   0.18804048  0.17316614 -0.354452  ]
 [ 0.04028251  0.30192393  1.05466831  0.99216727 -0.67128334]
 [-0.14006993  0.06545506  0.38196629  0.04395383  0.09118204]
 [-1.02208798 -0.41597551  0.64733441  0.70925579 -0.39137275]
 [ 0.06883237  0.32369845 -0.47307993 -0.61482263 -0.05680422]
 [ 0.66318189  0.28705839  0.47475443 -0.59275355 -0.1116078 ]
 [-0.77662379 -0.34958357  0.2403412   0.52315108  0.17215894]]
[[-0.5254357 ]
 [ 0.63601096]
 [-0.46425612]
 [ 1.11622721]
 [-1.0904324 ]
 [-1.11025381]
 [ 0.38449044]
 [-1.13920051]
 [-0.00328574]
 [-0.91974108]]


In [288]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [334]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 10000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 10000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 0.017717554561658775
----------cost is ------------- 0.017687843258119545
----------cost is ------------- 0.017660248073648622
----------cost is ------------- 0.01763405258625097
----------cost is ------------- 0.01760888248265209
----------cost is ------------- 0.017584529934112417
----------cost is ------------- 0.017560869843031797
----------cost is ------------- 0.017537819590968243
----------cost is ------------- 0.01751531946117292
----------cost is ------------- 0.017493322936981425
----------cost is ------------- 0.017471791753348256
----------cost is ------------- 0.017450693262534702
----------cost is ------------- 0.01742999894673853
----------cost is ------------- 0.01740968351482111
----------cost is ------------- 0.017389724308571417
----------cost is ------------- 0.017370100882189488
----------cost is ------------- 0.017350794685486372
----------cost is ------------- 0.01733178881397196
----------cost is ------------- 0.01731306780527064


----------cost is ------------- 0.01563225047964529
----------cost is ------------- 0.015623422843558192
----------cost is ------------- 0.015614618661688935
----------cost is ------------- 0.015605837765105832
----------cost is ------------- 0.015597079987015072
----------cost is ------------- 0.015588345162720249
----------cost is ------------- 0.015579633129582797
----------cost is ------------- 0.015570943726983522
----------cost is ------------- 0.015562276796285078
----------cost is ------------- 0.015553632180795427
----------cost is ------------- 0.015545009725732204
----------cost is ------------- 0.015536409278188027
----------cost is ------------- 0.015527830687096523
----------cost is ------------- 0.015519273803199389
----------cost is ------------- 0.015510738479014144
----------cost is ------------- 0.015502224568802694
----------cost is ------------- 0.015493731928540621
----------cost is ------------- 0.015485260415887343
----------cost is ------------- 0.0154768098901

----------cost is ------------- 0.014447405341444454
----------cost is ------------- 0.014441007304884757
----------cost is ------------- 0.01443461941853388
----------cost is ------------- 0.014428241642218037
----------cost is ------------- 0.014421873936035027
----------cost is ------------- 0.014415516260351662
----------cost is ------------- 0.014409168575801254
----------cost is ------------- 0.014402830843281122
----------cost is ------------- 0.0143965030239501
----------cost is ------------- 0.014390185079226134
----------cost is ------------- 0.01438387697078383
----------cost is ------------- 0.014377578660552124
----------cost is ------------- 0.014371290110711865
----------cost is ------------- 0.014365011283693555
----------cost is ------------- 0.014358742142175014
----------cost is ------------- 0.014352482649079108
----------cost is ------------- 0.014346232767571538
----------cost is ------------- 0.014339992461058573
----------cost is ------------- 0.0143337616931849

----------cost is ------------- 0.0135211548781817
----------cost is ------------- 0.013516014861656394
----------cost is ------------- 0.01351088082925913
----------cost is ------------- 0.013505752764834495
----------cost is ------------- 0.013500630652302672
----------cost is ------------- 0.013495514475658851
----------cost is ------------- 0.013490404218972782
----------cost is ------------- 0.013485299866388297
----------cost is ------------- 0.01348020140212278
----------cost is ------------- 0.013475108810466692
----------cost is ------------- 0.013470022075783118
----------cost is ------------- 0.013464941182507236
----------cost is ------------- 0.013459866115145932
----------cost is ------------- 0.013454796858277246
----------cost is ------------- 0.013449733396549967
----------cost is ------------- 0.013444675714683118
----------cost is ------------- 0.01343962379746556
----------cost is ------------- 0.0134345776297555
----------cost is ------------- 0.013429537196480055


----------cost is ------------- 0.012757004936916349
----------cost is ------------- 0.012752665287359131
----------cost is ------------- 0.012748329730052944
----------cost is ------------- 0.012743998256632388
----------cost is ------------- 0.012739670858761563
----------cost is ------------- 0.012735347528133914
----------cost is ------------- 0.012731028256472047
----------cost is ------------- 0.012726713035527628
----------cost is ------------- 0.012722401857081195
----------cost is ------------- 0.01271809471294205
----------cost is ------------- 0.01271379159494807
----------cost is ------------- 0.012709492494965616
----------cost is ------------- 0.012705197404889334
----------cost is ------------- 0.012700906316642061
----------cost is ------------- 0.012696619222174672
----------cost is ------------- 0.012692336113465912
----------cost is ------------- 0.012688056982522298
----------cost is ------------- 0.012683781821377938
----------cost is ------------- 0.01267951062209

KeyboardInterrupt: 

In [380]:
### 先用训练集验证

index = random.randint(0,999)
x_train_test = x[index]
x_train_test = x_train_test.reshape(1,-1)
#x_train_test = np.insert(x_train_test,0,1,axis=0)
y_test = y[index]
print('--序号--',index)
print('----输入-----',x_train_test)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('-----输出-----',y_test,hpfs_output)

--序号-- 300
----输入----- [[ 1.      -1.4755  -0.95059 -1.02261 -1.60917  1.25669  1.26116 -1.29563
  -1.32513 -0.27301  1.48132  1.54959 -0.54268 -1.26893 -1.02261 -0.28358
  -0.87587 -0.5254  -0.86829 -0.76169 -1.21218  0.78655  0.71456 -0.45468
  -0.31392 -0.51894 -0.56057  0.37408  0.08883 -0.3749  -0.35867 -0.9064
   0.07908  0.07893  0.49574  1.25125  1.38129  0.88372  0.94808  1.08178
   0.43706  0.29868  0.12013 -0.36574 -0.39467  1.23496  0.6411   0.71735
   0.82437 -1.13849 -0.69031  0.93433  0.80734  0.99377 -0.78514 -0.85581
   0.74215  0.6411   0.46552  0.74523 -0.56449 -1.21605  1.16743  1.33948
   1.2777  -1.19607 -1.09261]]
-----输出----- [1] [[0.99994778]]


In [406]:
#index = random.randint(1501,2100)
error_data = pd.DataFrame(columns =['1','2','3'])
#index = 300
for index in range(1500,2106):
    x_test = stock_data.loc[index*6:index*6+5].values
    x_test = x_test.reshape(1,-1)
    x_test = np.insert(x_test,0,1,axis=1)
    print('--序号--',index)
    #print('----输入-----',x_test)
    y_test = stock_lable.loc[index,2]
    hid_input = np.matmul(x_test,w1)
    hid_output = np.tanh(hid_input)
    hid_output = np.insert(hid_output,0,1,axis=1)
    hpfs_input = np.matmul(hid_output,w2)
    hpfs_output = sigmond(hpfs_input)
    output = hpfs_output[0][0]
    y_value = y_test
    #print(output)
    if(output > 0.5 and y_value ==1) or (output<0.5 and y_value ==0):
        print('-----输出-----',y_test,hpfs_output)
    else:
        print('-----输出-----',y_test,hpfs_output)
        error_data.append(stock_lable.loc[index],ignore_index='True')
    #print('-----输出-----',y_test,hpfs_output)

--序号-- 1500
-----输出----- 0 [[0.00656262]]
--序号-- 1501
-----输出----- 1 [[0.92865294]]
--序号-- 1502
-----输出----- 1 [[0.90934953]]
--序号-- 1503
-----输出----- 1 [[0.93209447]]
--序号-- 1504
-----输出----- 1 [[0.68423193]]
--序号-- 1505
-----输出----- 0 [[0.03657209]]
--序号-- 1506
-----输出----- 1 [[0.98012878]]
--序号-- 1507
-----输出----- 1 [[0.88742524]]
--序号-- 1508
-----输出----- 0 [[6.44272962e-07]]
--序号-- 1509
-----输出----- 0 [[0.00626747]]
--序号-- 1510
-----输出----- 0 [[0.22536968]]
--序号-- 1511
-----输出----- 0 [[0.00370297]]
--序号-- 1512
-----输出----- 0 [[0.0243297]]
--序号-- 1513
-----输出----- 0 [[0.02455137]]
--序号-- 1514
-----输出----- 0 [[1.21183887e-05]]
--序号-- 1515
-----输出----- 0 [[0.06097726]]
--序号-- 1516
-----输出----- 1 [[0.96687509]]
--序号-- 1517
-----输出----- 0 [[1.50409283e-05]]
--序号-- 1518
-----输出----- 1 [[0.92723034]]
--序号-- 1519
-----输出----- 0 [[0.00824368]]
--序号-- 1520
-----输出----- 0 [[9.70175073e-06]]
--序号-- 1521
-----输出----- 0 [[0.0208912]]
--序号-- 1522
-----输出----- 0 [[2.70032479e-07]]
--序号-- 1523
----

-----输出----- 0 [[0.00039818]]
--序号-- 1820
-----输出----- 0 [[0.00279974]]
--序号-- 1821
-----输出----- 1 [[0.83252973]]
--序号-- 1822
-----输出----- 0 [[0.1872258]]
--序号-- 1823
-----输出----- 1 [[0.7841911]]
--序号-- 1824
-----输出----- 0 [[0.00018657]]
--序号-- 1825
-----输出----- 0 [[0.00523285]]
--序号-- 1826
-----输出----- 0 [[0.0552094]]
--序号-- 1827
-----输出----- 0 [[0.01744112]]
--序号-- 1828
-----输出----- 1 [[0.96743895]]
--序号-- 1829
-----输出----- 0 [[0.00051674]]
--序号-- 1830
-----输出----- 1 [[0.97238714]]
--序号-- 1831
-----输出----- 1 [[0.96420444]]
--序号-- 1832
-----输出----- 0 [[0.00072013]]
--序号-- 1833
-----输出----- 1 [[0.96699132]]
--序号-- 1834
-----输出----- 1 [[0.96663597]]
--序号-- 1835
-----输出----- 0 [[3.02833991e-07]]
--序号-- 1836
-----输出----- 0 [[0.0070938]]
--序号-- 1837
-----输出----- 0 [[0.00111843]]
--序号-- 1838
-----输出----- 0 [[0.0044879]]
--序号-- 1839
-----输出----- 0 [[0.04903201]]
--序号-- 1840
-----输出----- 1 [[0.93935008]]
--序号-- 1841
-----输出----- 0 [[0.00436822]]
--序号-- 1842
-----输出----- 1 [[0.89566911]]
--序号-

In [407]:
print(error_data.shape)

(0, 3)


In [409]:
print(error_data.values)

[]
